<a href="https://colab.research.google.com/github/MuhamedAdemi/Brainster-Python/blob/main/MuhamedAdemi_Workshop_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop #3: Multivariable Calculus

In [ ]:
#importing libraries
import numpy as np
import sympy as sp
import scipy.optimize as opt
import scipy.integrate as spi
from scipy.optimize import minimize

## Problem 1
Find the critical points of the function $f(x, y) = x^2 + y^4 - 4xy$ by using first partial derivatives. Then use second partial derivatives to establish whether each critical point is a local minimum, a local maximum, or a saddle point.

In [ ]:
# Define the variables and the function
x, y = sp.symbols('x y')
f = x**4 + y**4 - 4*x*y + 1

# Differentiate
f_x = sp.diff(f, x)
f_y = sp.diff(f, y)

# Find critical points
critical_points = sp.solve([f_x, f_y], (x, y), dict=True)
critical_points = [(cp[x], cp[y]) for cp in critical_points]  # Convert to tuple list
print("Critical Points:", critical_points)

# Classifying the critical points
# Getting the second derivatives
f_xx = sp.diff(f_x, x)
f_xy = sp.diff(f_x, y)
f_yy = sp.diff(f_y, y)

# Function to classify a critical point
def classify_critical_point(cp, f_xx, f_xy, f_yy):
    D = f_xx.subs({x: cp[0], y: cp[1]}) * f_yy.subs({x: cp[0], y: cp[1]}) - (f_xy.subs({x: cp[0], y: cp[1]}))**2
    f_xx_val = f_xx.subs({x: cp[0], y: cp[1]})
    if D < 0:
        return "Saddle Point"
    elif D > 0:
        return "Local Minimum" if f_xx_val > 0 else "Local Maximum"
    else:
        return "Inconclusive"

# Working with the first critical point
cp1 = critical_points[0]
classification_cp1 = classify_critical_point(cp1, f_xx, f_xy, f_yy)
print(f"First Critical Point: {cp1} - {classification_cp1}")

# Working with the second critical point
cp2 = critical_points[1]
classification_cp2 = classify_critical_point(cp2, f_xx, f_xy, f_yy)
print(f"Second Critical Point: {cp2} - {classification_cp2}")

# Working with the third critical point
cp3 = critical_points[2]
classification_cp3 = classify_critical_point(cp3, f_xx, f_xy, f_yy)
print(f"Third Critical Point: {cp3} - {classification_cp3}")


Critical Points: [(-1, -1), (0, 0), (1, 1), (-I, I), (I, -I), (sqrt(2)*(-1 - I)/2, sqrt(2)/2 - sqrt(2)*I/2), (sqrt(2)*(-1 + I)/2, sqrt(2)/2 + sqrt(2)*I/2), (sqrt(2)*(1 - I)/2, -sqrt(2)/2 - sqrt(2)*I/2), (sqrt(2)*(1 + I)/2, -sqrt(2)/2 + sqrt(2)*I/2)]
First Critical Point: (-1, -1) - Local Minimum
Second Critical Point: (0, 0) - Saddle Point
Third Critical Point: (1, 1) - Local Minimum


## Problem 2
Using the **Gradient Descent Method** with initial approximation $\mathbf{x}_0 = (0, 0)$, find the minimum point and the minimum value of the function $g(x, y) = (1 - x + x^2)\cdot e^{y^2} + (1 - y + y^2)\cdot e^{x^2}$.

In [ ]:
# Define variables and functions
def g(x):
    return (1 - x[0] + x[0]**2) * np.exp(x[1]**2) + (1 - x[1] + x[1]**2) * np.exp(x[0]**2)

def grad_g(x):
    df_dx = (-1 + 2*x[0]) * np.exp(x[1]**2) + 2*x[0] * (1 - x[1] + x[1]**2) * np.exp(x[0]**2)
    df_dy = 2*x[1] * (1 - x[0] + x[0]**2) * np.exp(x[1]**2) + (-1 + 2*x[1]) * np.exp(x[0]**2)
    return np.array([df_dx, df_dy])

# Running the Gradient Descent Method
def gradient_descent(f, grad, x0, alpha=0.01, mode='min', max_iter=1000, tol=1e-6):
    x = x0.copy()
    for i in range(max_iter):
        gradient = grad(x)
        if np.linalg.norm(gradient) < tol:
            break
        if mode == 'min':
            x = x - alpha * gradient
        elif mode == 'max':
            x = x + alpha * gradient
    return x, f(x)

# Initial parameters
x0 = np.array([0.0, 0.0])
alpha = 0.01  # Learning rate

# Run Gradient Descent
min_point, min_value = gradient_descent(g, grad_g, x0, alpha=alpha, mode='min', max_iter=10000)

print(f"Minimum Point: ({min_point[0]:.6f}, {min_point[1]:.6f})")
print(f"Minimum Value: {min_value:.6f}")


Minimum Point: (0.277880, 0.277880)
Minimum Value: 1.727011


## Problem 3
On a certain workday, the *rate*, in tons per hour, at which unprocessed gravel arrives at a gravel processing plant is modeled by $G(t) = 90 + 45\cdot \cos⁡\left(\frac{t^2}{18}\right)$, where $t$ is measured in hours and $0 \leqslant t \leqslant 8$. At the beginning of the workday, $t = 0$, the plant has 500 tons of unprocessed gravel. During the hours of operation, $0 \leqslant t \leqslant 8$, the plant processes gravel at a constant rate $P(t) = 100$ tons per hour.
* Find the total amount of unprocessed gravel that arrives at the plant during the hours of operation on this workday.
* Is the amount of unprocessed gravel at the end of the workday (t=8) greater or smaller than amount of gravel at the beginning of the workday?

In [ ]:
# Define the rate function G(t)
def G(t):
    return 90 + 45 * np.cos(t**2 / 18)

# First bullet point: Total unprocessed gravel arrived during 0 ≤ t ≤ 8
total_arrived, error = spi.quad(G, 0, 8)
print(f"Total arrived gravel: {total_arrived:.2f} tons")

# Second bullet point: Compare final unprocessed gravel with initial 500 tons
initial_gravel = 500
processed_total = 100 * 8  # P(t) = 100 tons/hour * 8 hours
final_gravel = initial_gravel + total_arrived - processed_total

if final_gravel > initial_gravel:
    conclusion = "greater"
else:
    conclusion = "smaller"

print(f"Final unprocessed gravel at t=8: {final_gravel:.2f} tons")
print(f"The amount at t=8 is {conclusion} than the initial 500 tons.")


Total arrived gravel: 825.55 tons
Final unprocessed gravel at t=8: 525.55 tons
The amount at t=8 is greater than the initial 500 tons.


## Problem 4
Solve the system of equations:
\begin{equation}
\left\{
\begin{array}{rcl}
x - 2y + 3z &=& -1\\
3x + 2y - 5z &=& 3\\
2x - 5y + 2z &=& 0
\end{array}
\right.
\end{equation}

using **Gradient Descent Method** applied to a function of the kind $f(x) = \|A\mathbf{x} - b\|_2^2$ where $A\mathbf{x} = b$ is the matrix equation that corresponds to the system, and $\| \cdot \|_2$ is the Euclidean norm.


In [ ]:
# Define the matrix A and vector b from the system Ax = b
A = np.array([[1, -2, 3],
              [3, 2, -5],
              [2, -5, 2]], dtype=np.float64)
b = np.array([-1, 3, 0], dtype=np.float64)

# Define the residual squared function and its gradient
def f(x):
    residual = A @ x - b
    return np.sum(residual ** 2)

def grad_f(x):
    return 2 * A.T @ (A @ x - b)  # Gradient: 2Aᵀ(Ax - b)

# Gradient Descent Implementation
def gradient_descent(f, grad, x0, alpha=0.001, max_iter=10000, tol=1e-6):
    x = x0.copy()
    for _ in range(max_iter):
        gradient = grad(x)
        if np.linalg.norm(gradient) < tol:  # Check convergence
            break
        x -= alpha * gradient  # Update step
    return x, f(x)

# Initial guess and parameters
x0 = np.zeros(3)  # Start at [0, 0, 0]
alpha = 0.001     # Learning rate (adjust empirically)

# Run Gradient Descent
solution, min_value = gradient_descent(f, grad_f, x0, alpha=alpha)
print(f"Solution via Gradient Descent: {solution}")
print(f"Residual squared: {min_value:.6f}")

# Verify with direct linear solver
direct_solution = np.linalg.solve(A, b)
print(f"Exact solution (np.linalg.solve): {direct_solution}")


Solution via Gradient Descent: [ 0.26086976 -0.08695634 -0.4782607 ]
Residual squared: 0.000000
Exact solution (np.linalg.solve): [ 0.26086957 -0.08695652 -0.47826087]


## Problem 5

[Use Gradient Descent Method] Four people are typing two paragraphs of text. The first paragraph uses elementary vocabulary, while the second paragraph contains some more advanced words. The number of typos that the people make are labeled by 𝑥, for the first paragraph, and 𝑦, for the second paragraph. Data collected is given in the accompanying table:

| x 	 | y 	|
|---	 |---	|
| 1 	 | 4 	|
| 3 	 | 5 	|
| 4 	 | 6 	|
| 5 	 | 8 	|


We will build a model to predict the number of typos $y$ in the second text based on the number of typos in the first test. To do this, we use least-squares regression, an approach similar to the one we used for systems. For every number of observed typos $x_i$ for the first text, we have a corresponding number $y_i$ of observed typos for the second text. Our model $\hat{y_i} = f(x_i)$ will make a prediction of the number of typos on the second text. Most often, the observed number of typos $y_i$ and the predicted number of typos $\hat{y_i}$ will not be equal.
Let us label each such discrepancy in values by $d_i$, in other words:

\begin{align}
d_i = observed_i - predicted_i = y_i - \hat{y_i}
\end{align}


We call these discrepancies residuals. The goal now is to choose a model which will minimize the total sum of the squares of the residuals, i.e. a model which will make the overall discrepancy between the observed data and the predictions based on it as small as possible. Note that the form of the residuals depends on the choice of the model. For example, choosing a linear model $\hat{y_i}=a+bx$, we have prediction $\hat{y_i}=a+bx$, so the residuals become:

\begin{align}
d^2 = (y-\hat{y})^2 = (y-a-bx)^2
\end{align}

The total sum of the squares of the residuals is then $\sum_{i=1}^n d_i^2 = \sum_{i=1}^n (y_i-a-bx_i)^2$. This allows us to define the function $f$ that we need to minimize in order to build the model as:

\begin{align}
f(a,b) = \sum_{i=1}^n(y_i-a-bx_i)^2
\end{align}

By minimizing this function, we find the values for the coefficients $a$ and $b$ in the model that minimize the discrepancy between the data and the type of model we chose to work with.

*a)* Build a linear model for the data: $\hat{y}=a+bx$. Then make a prediction about the number of typos $y$ a person would make when typing the second text if they have made $x=2$ typos when typing the first text. **Use your own gradient_descent function.**

*b)* Build a quadratic model for the data: $\hat{y}=a+bx+cx^2$. Then make a prediction about the same person from part a). **Use the minimizer BFGS.**

**a) Linear model:**

In [ ]:
import numpy as np

# Data points (x, y)
X = np.array([1, 3, 4, 5])  # Independent variable (number of typos in first paragraph)
Y = np.array([4, 5, 6, 8])  # Dependent variable (number of typos in second paragraph)

# Define the cost function (Mean Squared Error)
def f_func(params):
    m, b = params
    predictions = m * X + b
    return np.sum((predictions - Y) ** 2) / len(X)  # Mean Squared Error (MSE)

# Define the gradient function for gradient descent
def grad_f(params):
    m, b = params
    n = len(X)
    predictions = m * X + b
    dm = (-2/n) * np.sum(X * (Y - predictions))  # Partial derivative w.r.t m
    db = (-2/n) * np.sum(Y - predictions)        # Partial derivative w.r.t b
    return np.array([dm, db])


In [ ]:
# Initialize parameters (m and b)
params = np.random.randn(2)

# Gradient descent parameters
alpha = 0.01
epochs = 1000

# Perform Gradient Descent
for _ in range(epochs):
    params -= alpha * grad_f(params)

solution_vector = params
print(f"Optimal parameters (m, b): {solution_vector}")

# Prediction for x = 2
x_new = 2
y_pred = solution_vector[0] * x_new + solution_vector[1]
print(f"Predicted typos for x={x_new}: {y_pred}")


Optimal parameters (m, b): [0.96780827 2.58906494]
Predicted typos for x=2: 4.524681485644655


**b) Quadratic model:**

In [ ]:
# Define variables and matrices


# Define the function f


# Define the numpy function f

#def f_func(x0):
#   return
from scipy.optimize import minimize

# Define quadratic model y = a*x^2 + b*x + c
def f_func(params):
    a, b, c = params
    predictions = a * X**2 + b * X + c
    return np.sum((predictions - Y)**2)  # Sum of Squared Errors (SSE)


In [ ]:
# Initial guess for parameters [a, b, c]
initial_params = np.random.randn(3)

# Optimize using BFGS
result = minimize(f_func, initial_params, method='BFGS')

# Extract optimal parameters
solution_vector = result.x



In [ ]:
print(f"Optimal parameters (a, b, c): {solution_vector}")

# Prediction for x = 2
x_new = 2
y_pred_quad = solution_vector[0] * x_new**2 + solution_vector[1] * x_new + solution_vector[2]
print(f"Predicted typos for x={x_new}: {y_pred_quad}")


Optimal parameters (a, b, c): [ 0.27272692 -0.65454339  4.39999808]
Predicted typos for x=2: 4.181818968173459
